In [1]:
!pip install -q textx

In [34]:
import os
from collections import deque
import time
from nltk import CFG
from nltk.parse.generate import generate
os.chdir("/content/drive/MyDrive/X_ACL")

In [3]:
p_file = "suffixs_v2.txt"
r_file = "roots_v2.txt"
s_file = 'prefixs_v2.txt'

Explanation of notation

*   '+' -- Prefix (p)
*   '$' -- Root (r)
*   '-' -- Interfix (i)
*   '^' -- Suffix (s)
*   '*' -- Ending (e)
*   '@' -- Postfix (x)

*   '&', 'T' -- start and end of the word


#Mophemes

In [37]:
class Morphemes:
    def __init__(self, prefixes, roots, interfixes, suffixes, endings, postfixes):
        self.morphemes_dict = self.load_morphemes(prefixes, roots, interfixes, suffixes, endings, postfixes) #dict of morphemes {"prefixes":{...}, ...}
        #all_morphemes in one set
        self.morphemes = self.morphemes_dict["prefixes"] | self.morphemes_dict["roots"] | self.morphemes_dict["interfixes"] | self.morphemes_dict["suffixes"] | self.morphemes_dict["endings"] | self.morphemes_dict["postfixes"]

    def load_morphemes(self, prefixes, roots, interfixes, suffixes, endings, postfixes):
        morphemes_dict = {
            "prefixes": set(["+" + i for i in prefixes]),
            "roots": set(["$" + i for i in roots]),
            "interfixes": set(["-" + i for i in interfixes]),
            "suffixes": set(["^" + i for i in suffixes]),
            "endings": set(["*" + i for i in endings]),
            "postfixes": set(["@" + i for i in postfixes])
        }
        return morphemes_dict

#BNF parser (fails, inefficient) DO NOT RUN

In [60]:
from textx import metamodel_from_str
class BNF_parser:
  def __init__(self, M: Morphemes):
    self.M = M
    self.grammar = self.generate_grammar() # TextX metamodel 'mm'

  def list_to_bnf(self, lst):
    return ("|").join(["'" + i + "'" for i in lst])

  def generate_grammar(self):
    grammar = f"""
    Word:
        base=Base
        (end=End)?;

    Base:
        stem=Stem
        (xsuffix=XSuffix)?;

    Stem:
        (xprefix=XPrefix)?
        root=SuperRoot;

    XPrefix:
        prefix=Prefix
        (xprefix=XPrefix)?;

    SuperRoot:
        root=Root
        (xroot=XRoot)?;

    XRoot:
        (interfix=Interfix)?
        base=Base;

    XSuffix:
      suffix=Suffix
      (xsuffix=XSuffix)?;

    End:
      (ending=Ending)? (postfix=Postfix)?;

    Prefix:
        ({self.list_to_bnf(self.M.morphemes_dict["prefixes"])});

    Root:
        {self.list_to_bnf(self.M.morphemes_dict["roots"])};

    Interfix:
        {self.list_to_bnf(self.M.morphemes_dict["interfixes"])};

    Ending:
      {self.list_to_bnf(self.M.morphemes_dict["endings"])};

    Postfix:
      {self.list_to_bnf(self.M.morphemes_dict["postfixes"])};

    Suffix:
        ({self.list_to_bnf(self.M.morphemes_dict["suffixes"])});
    """
    return metamodel_from_str(grammar)

  def check_word(self, w):
    try:
      model = self.grammar.model_from_str(w)
      if model:
          print(model)
          print(f"{w} is a valid word according to the morphology")
      else:
          print(f"{w} is not a valid word according to the morphology")
    except:
      print(f"{w} is not a valid word according to the morphology")

  def print_model_tree(self, obj, indent=0):
    #print(" " * indent + f"<{obj.__class__.__name__}>")
    if obj is None:
        pass#print(" " * indent + "None")
    elif isinstance(obj, str):
        print(" " * indent + "" + obj)
    elif isinstance(obj, list):
        print(" " * indent, [i[:-1] for i in obj])
    else:
        if hasattr(obj, "__dict__"):
            for key, value in obj.__dict__.items():
              if key != "parent" and key[0] != "_":
                print(" "*(indent + 3), "-", key)
                self.print_model_tree(value, indent + 6)

  def tree_for_word(self, w):
  # Assuming you already have the TextX metamodel 'mm' and the input 'word' "+ви+пере$земл-е$трус^ити@ся"
    try:
      #print(word)
      model = self.grammar.model_from_str(w)

      return model
    except:
      #print("Such parsing cannot exist")
      return None

  def find_combinations(self, w, morphemes):
    result = []

    def backtrack(start, path):
        if start == len(w):
            result.append(path[:])
            return

        for morpheme in morphemes:
            if w.startswith(morpheme[1:], start):
                path.append(morpheme)
                backtrack(start + len(morpheme)-1, path)
                path.pop()

    backtrack(0, [])
    return result

  def print_possible_parsings(self, w):
    combinations = list({"".join(comb)
                    for comb
                    in self.find_combinations(w, self.M.morphemes)})
    print(combinations)
    print(len(combinations))


    for c in combinations:
      print(40*"-", c)
      self.print_model_tree(self.tree_for_word(c))

  def possible_parsings(self, word):
    res = []
    combinations = list({"".join(comb)
                    for comb
                    in self.find_combinations(word, self.M.morphemes)})
    for c in combinations:
      t = self.tree_for_word(c)
      if t:
        res.append(t)
    return res

# Linearizing BNF (more efficient)

In [80]:
#Define your grammar from string
#You can define it using other methods, but I only know this xD
grammar = CFG.fromstring("""
  W -> Base End | Base
  Base -> Stem | Stem Suffix
  Stem -> Prefix Root | Root
  Suffix -> "s" | XSuffix
  XSuffix -> "s" Suffix
  Prefix -> "p" | XPrefix
  XPrefix -> "p" Prefix
  End -> "e" | "ex" | "x"
  Root -> "r" | "r" XRoot
  XRoot -> "i" Base | Base  """)
#With this we "create" all the possible combinations
grammar.productions()

#Here you can see all the productions (sentences) with 5 words
#created with this grammar
possible_sequences = []
i = 0
for production in generate(grammar, depth = 10):
  possible_sequences.append(''.join(production))
  i += 1
possible_sequences = list(set(possible_sequences))
len(possible_sequences)

528

#Mealy Machine Parser (backtracing -- pretty efficient)

In [7]:
class MealyMachine:
    def __init__(self, word, M:Morphemes):
        self.word = word
        self.transitions = self.fast_parse()
        self.initial_state = word[0] + '0'
        self.terminal_state = 'T'
        self.allowed_paths = {"&":{"p", "r"},
                              "p":{"p", "r"},
                              "r":{"p", "r", "T", "s", "i", "e"},
                              "i":{"p", "r"},
                              "s":{"s", "e", "x", "T", "i", "p"},
                              "e":{"x", "T"},
                              "x":{"T"},
                              "T":{"T"}}

    def num_letter_2_word(self, num_lettered_word: list):
      """["n1", "m2"] -> nm"""
      return "".join([i[0] for i in num_lettered_word])

    def morphemes_by_sequence(self, seqs):
      """перед -> {p, r}"""
      result = set()
      if "+" + seqs in M.morphemes:
        result.add("p")
      if "$" + seqs in M.morphemes:
        result.add("r")
      if "-" + seqs in M.morphemes:
        result.add("i")
      if "^" + seqs in M.morphemes:
        result.add("s")
      if "*" + seqs in M.morphemes:
        result.add("e")
      if "@" + seqs in M.morphemes:
        result.add("x")
      return result

    def word2matrix(self, word):
      letters_to_check = [word[i]+str(i) for i in range(len(word))]
      letter_matrix = {}
      for i in range(len(letters_to_check)):
        letter_matrix[letters_to_check[i]] = list()

      indicies_to_check = [0]
      while len(indicies_to_check) != 0:
        for index in indicies_to_check:
          indicies_to_check = []
          for i in range(len(letters_to_check)+1):
            if index > len(word):
              break
            combs = self.morphemes_by_sequence(word[index:i])
            if combs != set():
              combs = ([j + str(i-1) for j in list(combs)])
              letter_matrix[letters_to_check[index]] = letter_matrix[letters_to_check[index]]+([(letters_to_check[i-1],combs)])
              indicies_to_check.append(i)
      sequence = {letters_to_check[i]:letters_to_check[i+1] for i in range(len(letters_to_check) - 1)}
      sequence[letters_to_check[-1]] = "T"
      return letter_matrix, sequence, letters_to_check

    def fast_parse(self):
      word = self.word
      matrix = self.word2matrix(word)
      transitions = {}
      for letter in matrix[2]:
          if matrix[0][letter] == []:
            transitions[letter] = {}

          for trans in matrix[0][letter]:
            temp = dict()
            for morph in trans[1]:
              temp[morph] = {"output": self.num_letter_2_word(matrix[2][int(letter[1:]):int(trans[0][1:])+1]), "next_state":matrix[1][trans[0]]}
            if letter in transitions:
              transitions[letter].update(temp)
            else:
              transitions[letter] = temp
      transitions['T'] = {'/n': {'output': '/n', 'next_state': 'T'}}
      return transitions

    def get_possible_inputs(self, from_state, to_state):
        possible_inputs = []
        for symbol, transitions in self.transitions[from_state].items():
            if transitions['next_state'] == to_state:
                possible_inputs.append(symbol)
        return possible_inputs

    def backtrace_paths(self, current_state, terminal_state, current_output="", path=[], permitted_transitions = "&"):
        if current_state == terminal_state:
            return [(path + [(current_state, current_output)])]

        paths = []
        for symbol, transitions in self.transitions[current_state].items():

            next_state = transitions['next_state']
            output = transitions['output']

            possible_inputs = self.get_possible_inputs(current_state, next_state)
            for input_symbol in possible_inputs:

                #print(permitted_transitions)
                x = self.allowed_paths.get(permitted_transitions)
                #print(permitted_transitions, input_symbol[0], x)
                if input_symbol[0] in x:
                  new_path = path + [(current_state, input_symbol, output)]
                  new_output = current_output + output
                  paths.extend(self.backtrace_paths(next_state, terminal_state, new_output, new_path, permitted_transitions=input_symbol[0]))

        return paths

    def remove_duplicates(self, list_of_lists):
      list_of_tuples = [tuple(inner_list) for inner_list in list_of_lists]
      unique_set = set(list_of_tuples)
      unique_list_of_lists = [list(unique_tuple) for unique_tuple in unique_set]
      return unique_list_of_lists

    def all_paths(self):
      all_paths = self.backtrace_paths(self.initial_state, self.terminal_state)
      #print(all_paths)
      return self.remove_duplicates(all_paths)

# Filter using linearized BNF

In [8]:
class Filter:
  def __init__(self, word, M:Morphemes, morph_seqs):
    self.word = word
    self.mealy = MealyMachine(word, M)
    self.automata = self.mealy.fast_parse()
    self.graph = self.automata2graph()

  def automata2graph(self):
    """
    { 'з0': {'r2': {'output': 'зем', 'next_state': 'л3'},
      'r3': {'output': 'земл', 'next_state': 'е4'}},
      'е1': {},
      'м2': {},
      'л3': {},
      'е4': {'i4': {'output': 'е', 'next_state': 'T'},
        'e4': {'output': 'е', 'next_state': 'T'}},
      'T': {'/n': {'output': '/n', 'next_state': 'T'}}}

      ------>

    {'з0': {'л3': ['r2'], 'е4': ['r3']},
    'е1': {},
    'м2': {},
    'л3': {},
    'е4': {'T': ['i4', 'e4']},
    'T': {'T': ['/n']}}
    """
    graph = dict()
    for k in self.automata.keys():
      temp = dict()
      for m in self.automata[k]:
        temp[self.automata[k][m]['next_state']] = []
      for m in self.automata[k]:
        temp[self.automata[k][m]['next_state']].append(m)
      graph[k] = (temp)
    return(graph)

  def find_matching_paths(self, chars):
    start = self.word[0] + '0'
    queue = deque([(start, [])])
    paths = []

    while queue:
        node, path = queue.popleft()

        for neighbor in self.graph[node]:
            edges = self.graph[node][neighbor]
            for edge in edges:
                if ''.join(char for char in edge if char.isalpha()) == chars[len(path)]:
                    new_path = path + [neighbor]
                    if len(new_path) == len(chars):
                        paths.append([start] + new_path)
                    else:
                        queue.append((neighbor, new_path))

    return paths

  def inspect(self):
    res = []
    for chars in possible_sequences:
      chars = chars + 'n'
      graph = self.graph
      start = list(self.graph.keys())[0]
      if len(chars) < len(self.word)-1:
        if self.find_matching_paths(chars):
          res.append((chars, self.find_matching_paths(chars)))
    return res

# !Demo

##vocab

In [98]:
def preprocess_morph(lst):
  return [''.join(c for c in i if c.isalnum()) for i in lst]

In [99]:
prefixes =  ['пере', 'роз', 'за', 'про', 'при', 'ви', 'не', 'до', 'недо', "воз", "перед", "по", "що"] \
#+ preprocess_morph([line.strip() for line in open('prefixs_v2.txt', 'r')])
roots = ['мор', 'буд', 'зна', 'ход', 'від', 'мов', 'земл', 'дерев', 'жит', 'сел', 'вод', 'зем', 'кра', 'неб', 'міс', 'гір', 'ліс',
         'сон', 'дит', 'річ', 'мов', 'світ', 'голос', 'звук', 'тіл', 'серц', 'дух', 'люб', 'слов', 'час', 'рух', 'смаг', 'ліп',
         'люб', 'смаж', 'привіт', 'мандр', 'земл', 'трус', 'сп', 'свин', 'жер', 'вір', 'й', 'і', 'д', 'вз', "бач", "велич", "перед", "мокр", "год", "погод", "урок", "грош"]\
#+ preprocess_morph([line.strip() for line in open('roots_v2.txt', 'r')])
interfixes = ["о", "е"]
suffixes = ['ити', 'ти', "ть", 'ати', 'яти', 'ув', 'ство', 'к', 'анн',"ин", "ок", "ов"] \
#+ preprocess_morph([line.strip() for line in open('suffixs_v2.txt', 'r')])
endings = ["лю", "имо", "а", "я", "е", "ило", "у", "ий"]
postfixes = ["ся", "сь"]

In [100]:
M = Morphemes(prefixes, roots, interfixes, suffixes, endings, postfixes)

##BNF parser (very slow + errant) DO NOT RUN!

In [101]:
B = BNF_parser(M)
B.check_word("+що$земл-е$трус*у")

<textx:Word instance at 0x7d179634a320>
+що$земл-е$трус*у is a valid word according to the morphology


In [102]:
B.print_possible_parsings("щоземлетрусу")

['+що$земл*е$трус*у', '+що$земл-е$трус*у']
2
---------------------------------------- +що$земл*е$трус*у
---------------------------------------- +що$земл-е$трус*у
    - base
          - stem
                - xprefix
                      - prefix
                        +що
                      - xprefix
                - root
                      - root
                        $земл
                      - xroot
                            - interfix
                              -е
                            - base
                                  - stem
                                        - xprefix
                                        - root
                                              - root
                                                $трус
                                              - xroot
                                  - xsuffix
          - xsuffix
    - end
          - ending
            *у
          - postfix


In [103]:
#stress testing
start = time.time()
n = 100
for i in range(n):
  B = BNF_parser(M)
  B.possible_parsings("недопереземлетруситися")
print(f"Speed: {(time.time() - start)/n} per word")

Speed: 0.04900096893310547 per word


##Mealy Machine Parser

In [108]:
mealy = MealyMachine("недопереземлетруситися", M)
mealy.all_paths()

[[('н0', 'p1', 'не'),
  ('д2', 'p3', 'до'),
  ('п4', 'p7', 'пере'),
  ('з8', 'r11', 'земл'),
  ('е12', 'i12', 'е'),
  ('т13', 'r16', 'трус'),
  ('и17', 's19', 'ити'),
  ('с20', 'x21', 'ся'),
  ('T', 'недопереземлетруситися')],
 [('н0', 'p3', 'недо'),
  ('п4', 'p7', 'пере'),
  ('з8', 'r11', 'земл'),
  ('е12', 'i12', 'е'),
  ('т13', 'r16', 'трус'),
  ('и17', 's19', 'ити'),
  ('с20', 'x21', 'ся'),
  ('T', 'недопереземлетруситися')]]

In [109]:
#stress testing
start = time.time()
n = 100
for i in range(n):
  mealy = MealyMachine("недопереземлетруситися", M)
  mealy.all_paths()
print(f"Speed: {(time.time() - start)/n} per word")

Speed: 0.0008852481842041016 per word


## Mealy Parser + inspection by linearized BNF

In [110]:
f = Filter("недопереземлетруситися", M, possible_sequences)
f.inspect()

[('ppprirsxn',
  [['н0', 'д2', 'п4', 'з8', 'е12', 'т13', 'и17', 'с20', 'T', 'T']]),
 ('pprirsxn', [['н0', 'п4', 'з8', 'е12', 'т13', 'и17', 'с20', 'T', 'T']])]

In [111]:
#stress testing
start = time.time()
n = 100
for i in range(n):
  f = Filter("недопереземлетруситися", M, possible_sequences)
  f.inspect()
print(f"Speed: {(time.time() - start)/n} per word")

Speed: 0.016051878929138185 per word
